In [13]:
import sys
import ete3
import os
import random
from os import walk
sys.path.insert(0,'../scripts/')
from classDeclarationsAndFunctions import RootedTree
from fileIO import ReadRootedTree, ReadAlignment, WriteAlignment
from config import projectPath
# projectPath = 

In [4]:
# Create SARS-CoV-2 datasets for scalability
print(projectPath)
subsampled_nextstrain_sarscov2 = projectPath + "data/nextstrain/subsampled_21_2_2023/aligned.fasta"
full_subsampled_alignment = ReadAlignment(subsampled_nextstrain_sarscov2)




/home/kalaghat/exaptation/Projects/MSTBasedForests/


In [ ]:
seq_ids = list(full_subsampled_alignment.keys())
full_subsampled_alignment[seq_ids[0],seq_ids[1]]

In [ ]:
# create subsamples
from random import sample
seq_ids = list(full_subsampled_alignment.keys())

reps = 10
size_list = [1000,1500,2000,2500,3000]

for rep in range(reps):
    for size in size_list:
        resampled_ids = sample(seq_ids,size)
        resampled_aln = {}
        for seq_id in resampled_ids:
            resampled_aln[seq_id] = full_subsampled_alignment[seq_id]
        resampled_filename = projectPath+"data/nextstrain/subsampled_21_2_2023/resampled_"+str(size)+"_seqs_"+"rep_"+str(rep)+".fasta"
        WriteAlignment(resampled_aln,resampled_filename)
        print (size,rep)

In [ ]:
# create scripts for scalability
method_list = ['mstbackbone','fasttree','raxmlng','iqtree']
# Job scripts
data_path = projectPath + "data/"
tools_path_geniux = "/project/exaptation/Tools/"
iqtree_script = open("../batch_scripts/iqtree_sars_cov2_scalability.sh","w")
mstbackbone_script = open("../batch_scripts/mstbackbone_sars_cov2_scalability.sh","w")
fasttree_script = open("../batch_scripts/fastree_sars_cov2_scalability.sh","w")
raxml_script = open("../batch_scripts/raxml_sars_cov2_scalability.sh","w")

for rep in range(reps):
    for size in size_list:                
        mxqsub_prefix = ""        
        exp_id = "sars_cov2_size_"+str(size)+"_rep_"+str(rep)
        sequence_file_name = projectPath+"data/nextstrain/subsampled_21_2_2023/resampled_"+str(size)+"_seqs_"+"rep_"+str(rep)+".fasta"
        ###############        IQ-TREE      ##################
        # mxqsub_prefix = "mxqsub -t 2d -m 10G -n -N iqtree_scale\t"
        # mxqsub_prefix += " --stderr=err_iqtree_scalability_"+exp_id
        # mxqsub_prefix += " --stdout=out_iqtree_scalability_"+exp_id
        iqtree_command = tools_path_geniux + "iqtree2"
        iqtree_command += ' -s ' + sequence_file_name + ' -st DNA'
        iqtree_command += ' -nt ' + str(1)
        iqtree_command += ' -seed 1234'
        # iqtreeCommand += ' -quiet'
        iqtree_command += ' -redo'
        iqtree_command += ' -pre ' + sequence_file_name + "_iqtree2.0"
        iqtree_command += ' -m UNREST'
        iqtree_script.write(mxqsub_prefix + iqtree_command + "\n")
        ###############     MST-BACKBONE     ###################
        # mst_backbone_command
        # mxqsub_prefix = "mxqsub -t 2d -m 10G -n -N mstbackbone_scale\t"
        # mxqsub_prefix += " --stderr=err_mstbackbone_scalability_" + exp_id
        # mxqsub_prefix += " --stdout=out_mstbackbone_scalability_" + exp_id
        mst_backbone_command = tools_path_geniux + "mst-backbone"
        mst_backbone_command += '\t--seq ' + sequence_file_name
        mst_backbone_command += '\t--out ' + exp_id
        mstbackbone_script.write(mxqsub_prefix + mst_backbone_command + "\n")
        ###############        RAxML-NG       ##################
        # mxqsub_prefix = "mxqsub -t 2d -m 10G -n -N raxmlng_scale\t"
        # mxqsub_prefix += " --stderr=err_raxmlng_scalability_" + exp_id
        # mxqsub_prefix += " --stdout=out_raxmlng_scalability_" + exp_id
        raxml_command = tools_path_geniux + "raxml-ng"
        fileNamePrefix = 'RAxMLNG_' + exp_id
        raxml_command += ' --tree pars{1}'
        raxml_command += ' --model GTR --msa ' + sequence_file_name
        raxml_command += ' --threads 1'
        # raxml_command += ' --redo '
        raxml_command += ' --tip-inner on --pat-comp on --site-repeats on'
        raxml_command += ' --prefix ' + fileNamePrefix
        raxml_script.write(mxqsub_prefix + raxml_command + "\n")
        #############          FastTree        ################
        # mxqsub_prefix = ""
        # mxqsub_prefix = "mxqsub -t 2d -m 1G -n -N fasttree_scale\t"
        # mxqsub_prefix += " --stderr=err_fasttree_scalability_"+exp_id
        # mxqsub_prefix += " --stdout=out_fasttree_scalability_"+exp_id
        output_tree_file_name = sequence_file_name + '.fasttree_newick'
        std_err_file_name = sequence_file_name + '.fasttree_errlog'
        fasttree_command = tools_path_geniux + 'FastTree'
        fasttree_command += ' -nt -nosupport -nocat -gtr < '+ sequence_file_name
        fasttree_command += ' > ' + output_tree_file_name
        fasttree_command += ' 2> ' + std_err_file_name
        fasttree_script.write(mxqsub_prefix + fasttree_command + "\n")
        ########################################################



fasttree_script.close()
raxml_script.close()
mstbackbone_script.close()
iqtree_script.close()

In [1]:
def Get_run_time(method,size,rep):
    # print(method, exp_id)
    sequence_file_name = projectPath + "data/nextstrain/subsampled_21_2_2023/resampled_"+str(size)+"_seqs_"+"rep_"+str(rep)+".fasta"
    exp_id = "sars_cov2_size_"+str(size)+"_rep_"+str(rep)
    runTime = -1
    if method == "raxmlng":
        log_file_name = projectPath + '/batch_scripts/RAxMLNG_' + exp_id + '.raxml.log'
        log_file = open(log_file_name,"r")
        for line in log_file:
            if line.startswith('Elapsed time:'):
                runTime = float(line.strip().split('Elapsed time:')[1].split(' seconds')[0])
    elif method == "mstbackbone":
        log_file_name = projectPath + "data/nextstrain/subsampled_21_2_2023/resampled_"+str(size)+"_seqs_"+"rep_"+str(rep)+ '.mstbackbone_log'
        log_file = open(log_file_name,"r")
        for line in log_file:
            if line.startswith('Total CPU time used is '):
                runTime = float(line.split('Total CPU time used is ')[1].split(' second(s)')[0])    
    elif method == "fasttree":
        log_file_name = sequence_file_name + '.fasttree_errlog'
        log_file = open(log_file_name,"r")
        for line in log_file:
            if line.startswith('Total time: '):
                runTime = float(line.strip().split('Total time: ')[1].split(' seconds')[0])
    elif method == "iqtree":
        log_file_name = sequence_file_name + '_iqtree2.0.log'        
        log_file = open(log_file_name,"r")
        for line in log_file:
            if line.startswith('Total CPU time used:'):
                runTime = float(line.split('Total CPU time used: ')[1].split(' sec')[0])                
    log_file.close()
    return (runTime)

In [ ]:
# plot run times
method_list = ['mstbackbone']
for rep in range(reps):
    for size in size_list:
        for method in method_list:
            run_time = Get_run_time(method,size,rep)
        
